In [ ]:
import time
import random
import subprocess

import tskit
import numpy as np
import matplotlib.pyplot as plt
import tsinfer

the first line of simulation file tell us how was run the simulation

In [ ]:
subprocess.run(["head", "../test1M.out"])

In [ ]:
subprocess.run(["tail", "../test1M.out"])

I can use numpy to read data:

In [ ]:
dat = np.loadtxt("../test1M.out", skiprows=6)
type(dat)

get info on array content

In [ ]:
dat.dtype

In [ ]:
dat.shape

extract the geonotpe objects

In [ ]:
gts = np.array(dat[:, 2:], dtype=np.int16)
gts

In [ ]:
gts.shape

In [ ]:
plt.imshow(gts[:200], aspect="auto", cmap="viridis")
plt.show()

In [ ]:
relPos = dat[:, 0]

In [ ]:
plt.hist(relPos, bins=100)
plt.show()

In [ ]:
absPos = relPos * 1_000_000

I need big integers to represent absolute positions

In [ ]:
absPos = absPos.astype(np.int32)

In [ ]:
plt.hist(absPos, bins=100)
plt.show()

there could be position that are identical

In [ ]:
len(absPos.tolist())

In [ ]:
len(set(absPos.tolist()))

I need to create an object, I want to remove duplicates by adding one without change the original list:

In [ ]:
absPosShift = absPos.copy()
for i in range(1, absPos.shape[0]):
    if absPosShift[i] <= absPosShift[i - 1]:
        absPosShift[i] = absPosShift[i - 1] + 1

In [ ]:
len(set(absPosShift.tolist()))

## Inference with TSINFER
### set sampledata

In [ ]:
samp = tsinfer.SampleData(
    sequence_length=1_000_000,
    path="mySamples.samples")

add population information

In [ ]:
samp.add_population(metadata={"name": "Mouflon"})
samp.add_population(metadata={"name": "Iranian"})
samp.add_population(metadata={"name": "Border"})

In [ ]:
# this will create a list like of 105 elements [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...]
popID = np.repeat([0, 1, 2], [5, 50, 50]).tolist()
for i in range(105):
    samp.add_individual(ploidy=2, population=popID[i])

In [ ]:
list(samp.populations())

add variant data:

In [ ]:
for i in range(absPosShift.shape[0]):
    samp.add_site(absPosShift[i], gts[i, :])

Then finalize

In [ ]:
samp.finalise()

### TS inference

In [ ]:
t0 = time.time()
ts = tsinfer.infer(samp)
t1 = time.time()
t1 - t0

In [ ]:
ts = ts.simplify()
ts

In [ ]:
ts.at(110).draw_svg(size=(1000, 200))

In [ ]:
swapIndex = [random.random() < 0.1 for i in range(len(absPosShift))]

make a new sampledata object:

In [ ]:
samp2 = tsinfer.SampleData(
    sequence_length=1_000_000,
    path="mySamples2.samples"
)

In [ ]:
for i in range(len(absPosShift)):
    samp2.add_site(absPosShift[i], gts[i, :], ancestral_allele=int(swapIndex[i]))

samp2.finalise()

In [ ]:
ts2 = tsinfer.infer(samp2)

In [ ]:
ts2 = ts2.simplify()
ts2

In [ ]:
ts

ancestry are the same since they don't depends on trees

In [ ]:
ts.diversity([range(10), range(10, 110), range(110, 210)]) * 100

In [ ]:
ts2.diversity([range(10), range(10, 110), range(110, 210)]) * 100

also Fst

In [ ]:
ts.Fst([range(10), range(200, 210)])

In [ ]:
ts2.Fst([range(10), range(200, 210)])

nearest neighbours change a little:

In [ ]:
ts.genealogical_nearest_neighbours([0], [range(10), range(10, 110), range(110, 210)])

In [ ]:
ts2.genealogical_nearest_neighbours([0], [range(10), range(10, 110), range(110, 210)])

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(np.array(range(11))-.15, ts.simplify(range(10)).allele_frequency_spectrum(polarised=True), width=0.3, label="ts")
plt.bar(np.array(range(11))+.15, ts2.simplify(range(10)).allele_frequency_spectrum(polarised=True), width=0.3, label="ts2")
plt.legend()
plt.show()